<a href="https://colab.research.google.com/github/cevateness/set_partitioning_problem/blob/main/spp_algos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import itertools
import numpy as np

# Problem Data
elements = {1, 2, 3, 4, 5, 6}
subsets = [
    {"set": {1, 2}, "cost": 4},
    {"set": {2, 3, 4}, "cost": 6},
    {"set": {3, 5}, "cost": 5},
    {"set": {4, 6}, "cost": 7},
    {"set": {5, 6}, "cost": 3}
]

# Helper function to check if a solution is feasible (covers all elements exactly once)
def is_feasible(solution):
    covered_elements = set()
    for subset in solution:
        covered_elements.update(subset['set'])
    return covered_elements == elements and all(
        list(covered_elements).count(e) == 1 for e in elements
    )

# Brute-force approach (exhaustive search) to find the optimal solution
def brute_force_solution():
    min_cost = float('inf')
    best_solution = None
    for r in range(1, len(subsets) + 1):
        for combination in itertools.combinations(subsets, r):
            if is_feasible(combination):
                total_cost = sum(subset['cost'] for subset in combination)
                if total_cost < min_cost:
                    min_cost = total_cost
                    best_solution = combination
    return min_cost, best_solution

# Branch and Bound
def branch_and_bound():
    def bb_recursive(current_solution, remaining_subsets):
        # Base case: if the current solution is feasible
        if is_feasible(current_solution):
            return sum(subset['cost'] for subset in current_solution), current_solution

        # If no remaining subsets are left, return infinity cost
        if not remaining_subsets:
            return float('inf'), None

        # Branching: choose to include or exclude the next subset
        next_subset = remaining_subsets[0]
        cost_with, solution_with = bb_recursive(current_solution + [next_subset], remaining_subsets[1:])
        cost_without, solution_without = bb_recursive(current_solution, remaining_subsets[1:])

        # Return the better solution (with the minimum cost)
        if cost_with < cost_without:
            return cost_with, solution_with
        else:
            return cost_without, solution_without

    return bb_recursive([], subsets)

# Branch and Cut (simplified with separation of basic cuts)
def branch_and_cut():
    # This example uses a simplified branch-and-bound structure, adding basic cutting planes.
    # In a real branch-and-cut, we would generate cuts (e.g., violated constraints) dynamically.

    def separation_cut(solution):
        # Simple heuristic cut: if a solution covers more elements than needed, prune it.
        covered_elements = set()
        for subset in solution:
            covered_elements.update(subset['set'])
        if len(covered_elements) > len(elements):
            return True  # Indicates a cut is needed
        return False

    def bac_recursive(current_solution, remaining_subsets):
        if is_feasible(current_solution):
            return sum(subset['cost'] for subset in current_solution), current_solution
        if not remaining_subsets:
            return float('inf'), None

        next_subset = remaining_subsets[0]

        # Cut: prune branches with over-coverage
        if separation_cut(current_solution + [next_subset]):
            return float('inf'), None

        # Branching: include or exclude the next subset
        cost_with, solution_with = bac_recursive(current_solution + [next_subset], remaining_subsets[1:])
        cost_without, solution_without = bac_recursive(current_solution, remaining_subsets[1:])

        if cost_with < cost_without:
            return cost_with, solution_with
        else:
            return cost_without, solution_without

    return bac_recursive([], subsets)

# Branch and Price (simplified with basic column generation)
def branch_and_price():
    # In this simplified example, we manually generate columns (subsets) instead of dynamically.

    # Example pricing function: look for the cheapest new subset that improves the solution.
    def pricing(remaining_subsets):
        return min(remaining_subsets, key=lambda x: x['cost'])

    current_solution = []
    remaining_subsets = subsets[:]
    while not is_feasible(current_solution):
        new_subset = pricing(remaining_subsets)
        current_solution.append(new_subset)
        remaining_subsets.remove(new_subset)

    total_cost = sum(subset['cost'] for subset in current_solution)
    return total_cost, current_solution

# Solver-based approach using PuLP for comparison
from pulp import LpProblem, LpMinimize, LpVariable, lpSum

def solver_based_solution():
    problem = LpProblem("Set_Partitioning", LpMinimize)

    # Decision variables
    x = [LpVariable(f"x_{i}", cat="Binary") for i in range(len(subsets))]

    # Objective function
    problem += lpSum(x[i] * subsets[i]["cost"] for i in range(len(subsets)))

    # Constraints: cover each element exactly once
    for elem in elements:
        problem += lpSum(x[i] for i in range(len(subsets)) if elem in subsets[i]["set"]) == 1

    # Solve the problem
    problem.solve()

    solution = [subsets[i] for i in range(len(subsets)) if x[i].varValue == 1]
    total_cost = sum(subset["cost"] for subset in solution)
    return total_cost, solution

# Main Execution and Results
if __name__ == "__main__":
    print("Brute Force Solution:")
    min_cost, solution = brute_force_solution()
    print(f"Cost: {min_cost}, Solution: {solution}")

    print("\nBranch and Bound Solution:")
    min_cost, solution = branch_and_bound()
    print(f"Cost: {min_cost}, Solution: {solution}")

    print("\nBranch and Cut Solution:")
    min_cost, solution = branch_and_cut()
    print(f"Cost: {min_cost}, Solution: {solution}")

    print("\nBranch and Price Solution:")
    min_cost, solution = branch_and_price()
    print(f"Cost: {min_cost}, Solution: {solution}")

    print("\nSolver-based Solution:")
    min_cost, solution = solver_based_solution()
    print(f"Cost: {min_cost}, Solution: {solution}")


Brute Force Solution:
Cost: 13, Solution: ({'set': {1, 2}, 'cost': 4}, {'set': {2, 3, 4}, 'cost': 6}, {'set': {5, 6}, 'cost': 3})

Branch and Bound Solution:
Cost: 13, Solution: [{'set': {1, 2}, 'cost': 4}, {'set': {2, 3, 4}, 'cost': 6}, {'set': {5, 6}, 'cost': 3}]

Branch and Cut Solution:
Cost: 13, Solution: [{'set': {1, 2}, 'cost': 4}, {'set': {2, 3, 4}, 'cost': 6}, {'set': {5, 6}, 'cost': 3}]

Branch and Price Solution:
Cost: 18, Solution: [{'set': {5, 6}, 'cost': 3}, {'set': {1, 2}, 'cost': 4}, {'set': {3, 5}, 'cost': 5}, {'set': {2, 3, 4}, 'cost': 6}]

Solver-based Solution:
Cost: 16, Solution: [{'set': {1, 2}, 'cost': 4}, {'set': {3, 5}, 'cost': 5}, {'set': {4, 6}, 'cost': 7}]
